In [1]:
# ЯЧЕЙКА 2 — Импорты и настройки
import os
import random
import numpy as np
import pandas as pd
from tqdm.auto import tqdm
import torch

from sentence_transformers import SentenceTransformer, CrossEncoder, InputExample, losses
from torch.utils.data import DataLoader

# Воспроизводимость
random.seed(42)
np.random.seed(42)
torch.manual_seed(42)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(42)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Устройство: {device}")

Устройство: cpu


In [ ]:
# ЯЧЕЙКА 3 — Загрузка твоих данных
# Положи файлы в ту же папку с ноутбуком:
# gold_pairs.csv      → колонны: query, post, score (0.0–1.0)
# all_posts.csv       → колонна: post_text (и по желанию channel_name)

df_gold = pd.read_csv("gold_pairs.csv")
print(f"Золотых пар: {len(df_gold)}")
print(df_gold.head())

df_corpus = pd.read_csv("all_posts.csv")
posts = df_corpus['post_text'].astype(str).unique().tolist()
print(f"Уникальных постов в корпусе: {len(posts)}")

queries = df_gold['query'].unique().tolist()
print(f"Уникальных запросов: {len(queries)}")

In [ ]:
# ЯЧЕЙКА 4 — 1-й раунд: дообучение Cross-Encoder
print("Запускаем обучение Cross-Encoder...")

cross_model = CrossEncoder('cross-encoder/ms-marco-MiniLM-L-6-v2', num_labels=1, max_length=512)
cross_model.model.to(device)

train_examples = [
    InputExample(texts=[row['query'], row['post']], label=float(row['score']))
    for _, row in df_gold.iterrows()
]

train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=32)

cross_model.fit(
    train_dataloader=train_dataloader,
    epochs=4,
    warmup_steps=int(len(train_dataloader) * 4 * 0.1),
    output_path="cross_finetuned",
    show_progress_bar=True
)

print("Cross-Encoder дообучен и сохранён в папку cross_finetuned")

In [ ]:
# ЯЧЕЙКА 5 — Генерация серебряных меток (самый долгий этап)
print("Генерируем серебряные пары...")

cross_model = CrossEncoder("cross_finetuned")
cross_model.model.to(device)

SAMPLES_PER_QUERY = 150            # ← можно увеличить до 300–500, если есть время
pairs_to_score = []

for query in tqdm(queries, desc="Собираем пары"):
    sampled = random.sample(posts, k=min(SAMPLES_PER_QUERY, len(posts)))
    pairs_to_score.extend([(query, post) for post in sampled])

print(f"Всего пар для скоринга: {len(pairs_to_score):,}")

# Скорим большими батчами — быстрее
batch_size = 128
silver_scores = []

for i in tqdm(range(0, len(pairs_to_score), batch_size), desc="Скорим Cross-Encoder'ом"):
    batch = pairs_to_score[i:i+batch_size]
    scores = cross_model.predict(batch)
    silver_scores.extend(scores.tolist())

silver_df = pd.DataFrame({
    'query': [p[0] for p in pairs_to_score],
    'post':  [p[1] for p in pairs_to_score],
    'score': silver_scores
})

# Оставляем только уверенные метки
silver_df = silver_df[silver_df['score'].between(0.05, 0.95)]
print(f"Серебряных пар после фильтра: {len(silver_df):,}")
silver_df.to_csv("silver_pairs.csv", index=False)

In [ ]:
# ЯЧЕЙКА 6 — 2-й раунд: дообучение Bi-Encoder на золоте + серебре
print("Запускаем обучение финального Bi-Encoder...")

bi_model = SentenceTransformer('deepvk/RuModernBERT-small', revision="patched-tokenizer")
# альтернатива: SentenceTransformer('cointegrated/rubert-tiny2')

# Объединяем золото и серебро
all_pairs = pd.concat([df_gold[['query', 'post', 'score']], silver_df], ignore_index=True)
print(f"Всего пар для Bi-Encoder: {len(all_pairs):,}")

train_examples_bi = [
    InputExample(texts=[row['query'], row['post']], label=row['score'])
    for _, row in all_pairs.iterrows()
]

train_dataloader = DataLoader(train_examples_bi, shuffle=True, batch_size=64)
train_loss = losses.CosineSimilarityLoss(bi_model)

bi_model.fit(
    train_objectives=[(train_dataloader, train_loss)],
    epochs=3,
    warmup_steps=int(len(train_dataloader) * 3 * 0.1),
    output_path="bi_finetuned_final",
    show_progress_bar=True
)

print("ФИНАЛЬНАЯ МОДЕЛЬ ГОТОВА!")
print("Папка: bi_finetuned_final")
print("Используй в своём поиске так:")
print('bi_encoder = SentenceTransformer("bi_finetuned_final")')

In [ ]:
# ЯЧЕЙКА — Сохраняем финальную дообученную модель
bi_encoder_finetuned = bi_model  # или как у тебя называется переменная после обучения

bi_encoder_finetuned.save("bi_encoder_finetuned")
cross_model.save("cross_encoder_finetuned")       # ← ЭТУ СТРОКУ ДОБАВЬ!

print("Дообученная модель сохранена в папку: model_finetuned")